In [21]:
import cv2
import os
import yaml

# Helper function to convert YOLO format (normalized) to bounding box (x1, y1, x2, y2)
def yolo_to_bbox(x_center, y_center, width, height, img_width, img_height):
    x1 = int((x_center - width / 2) * img_width)
    y1 = int((y_center - height / 2) * img_height)
    x2 = int((x_center + width / 2) * img_width)
    y2 = int((y_center + height / 2) * img_height)
    return x1, y1, x2, y2

def tag_images(image_folder, label_folder, target_folder, class_names):
    # Create the target folder if it doesn't exist
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    print(f'Processing {image_folder} and {label_folder} to {target_folder}')

    # Iterate through the images and their corresponding labels
    for image_file in os.listdir(image_folder):
        if image_file.endswith(".jpg") or image_file.endswith(".png"):
            # Load image
            image_path = os.path.join(image_folder, image_file)
            image = cv2.imread(image_path)
            img_height, img_width = image.shape[:2]
    
            # Corresponding label file
            label_file = os.path.join(label_folder, image_file.replace('.jpg', '.txt').replace('.png', '.txt'))
    
            # Check if label file exists
            if os.path.exists(label_file):
                with open(label_file, 'r') as f:
                    for line in f.readlines():
                        class_id, x_center, y_center, width, height = map(float, line.split())
                        x1, y1, x2, y2 = yolo_to_bbox(x_center, y_center, width, height, img_width, img_height)
                        
                        # Draw the bounding box
                        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
                        # Draw the class label
                        label = f'{class_names[int(class_id)]}'
                        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
                # save the image with bounding boxes
                cv2.imwrite(target_folder + image_file, image)

def load_class_names(data_yaml_path):
    # load the class names from data.yaml
    with open(data_yaml_path, 'r') as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
        class_names = data['names']
    
    return class_names

def tag_yolo_dataset(dataset_path, output_path):
    # Load class names
    class_names = load_class_names(f'{dataset_path}\\data.yaml')
    
    print(f'Class names: {class_names}')
    
    subsets = ['train', 'val', 'test']
    
    if not os.path.exists(dataset_path + f'\\{output_path}'):
        os.makedirs(dataset_path + f'\\{output_path}')
    
    for subset in subsets:
        tag_images(f'{dataset_path}\\{subset}\\images', f'{dataset_path}\\{subset}\\labels', f'{dataset_path}\\{output_path}\\{subset}\\', class_names)

In [22]:
dataset_path = '..\\dataset\\yolo_signals_cbba'
tag_yolo_dataset(dataset_path, 'tagged_images')

Class names: {0: 'zona-escolar', 1: 'pare', 2: 'paso-peatonal', 3: 'ceda-el-paso', 4: 'limite-velocidad-10', 5: 'limite-velocidad-20', 6: 'limite-velocidad-30', 7: 'limite-velocidad-40', 8: 'limite-velocidad-35', 9: 'prohibido-girar-izquierda', 10: 'semaforo-rojo', 11: 'prohibido-girar-u', 12: 'semaforo-inteligente-rojo', 13: 'prohibido-girar-derecha', 14: 'pare-horizontal'}
Processing ..\dataset\yolo_signals_cbba\train\images and ..\dataset\yolo_signals_cbba\train\labels to ..\dataset\yolo_signals_cbba\tagged_images\train\
Processing ..\dataset\yolo_signals_cbba\val\images and ..\dataset\yolo_signals_cbba\val\labels to ..\dataset\yolo_signals_cbba\tagged_images\val\
Processing ..\dataset\yolo_signals_cbba\test\images and ..\dataset\yolo_signals_cbba\test\labels to ..\dataset\yolo_signals_cbba\tagged_images\test\
